In [77]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [78]:
df = pd.read_csv('../usdvnd/usdvnd_cleaned_features_classification.csv')
df

,Lần cuối,Mở,Cao,Thấp,% Thay đổi,Lần cuối 1,% Thay đổi 1,Lần cuối 3,Lần cuối 7,MA 7,MA 30,Std dev 7,RSI 14,Ngày thứ,Tháng thứ,Quý thứ,Năm thứ,Target,Ngày
0,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10981.0,10987.000000,10982.400000,2.645751,100.000000,5,9,3,1994,0,1994-09-03
1,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10981.0,10988.000000,10982.633333,0.000000,100.000000,6,9,3,1994,0,1994-09-04
2,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10982.866667,0.000000,100.000000,0,9,3,1994,0,1994-09-05
3,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10983.100000,0.000000,100.000000,1,9,3,1994,0,1994-09-06
4,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10983.333333,0.000000,100.000000,2,9,3,1994,0,1994-09-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11384,26312.0,26306.0,26326.5,26275.0,-0.01,26315.0,0.00,26315.0,26305.0,26321.571429,26335.866667,9.501879,35.121951,0,11,4,2025,0,2025-11-03
11385,26320.0,26316.0,26341.0,26302.5,0.03,26312.0,-0.01,26315.0,26328.0,26320.428571,26333.833333,9.071147,39.545455,1,11,4,2025,0,2025-11-04
11386,26322.5,26319.5,26345.0,26302.5,0.01,26320.0,0.03,26315.0,26336.0,26318.500000,26332.416667,6.184658,38.425926,2,11,4,2025,1,2025-11-05
11387,26315.0,26323.5,26338.0,26298.0,-0.03,26322.5,0.01,26312.0,26330.0,26316.357143,26330.750000,3.590662,41.500000,3,11,4,2025,1,2025-11-06


In [79]:
X = df.drop(columns=['Target', 'Ngày'])
y = df['Target']

In [80]:
split_point = int(len(X) * 0.8)

X_train, y_train = X.iloc[:split_point], y.iloc[:split_point]
X_test, y_test = X.iloc[split_point:], y.iloc[split_point:]

In [81]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [82]:
X_train

,Lần cuối,Mở,Cao,Thấp,% Thay đổi,Lần cuối 1,% Thay đổi 1,Lần cuối 3,Lần cuối 7,MA 7,MA 30,Std dev 7,RSI 14,Ngày thứ,Tháng thứ,Quý thứ,Năm thứ
0,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10981.0,10987.000000,10982.400000,2.645751,100.000000,5,9,3,1994
1,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10981.0,10988.000000,10982.633333,0.000000,100.000000,6,9,3,1994
2,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10982.866667,0.000000,100.000000,0,9,3,1994
3,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10983.100000,0.000000,100.000000,1,9,3,1994
4,10988.0,10988.0,10988.0,10988.0,0.00,10988.0,0.00,10988.0,10988.0,10988.000000,10983.333333,0.000000,100.000000,2,9,3,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106,23204.0,23204.0,23215.0,23199.5,-0.07,23220.0,0.02,23250.0,23215.0,23226.714286,23214.366667,23.164011,49.758454,4,8,3,2019
9107,23204.0,23204.0,23215.0,23199.5,0.00,23204.0,-0.07,23215.0,23215.0,23225.142857,23214.500000,24.429685,49.758454,5,8,3,2019
9108,23204.0,23204.0,23215.0,23199.5,0.00,23204.0,0.00,23220.0,23215.0,23223.571429,23214.633333,25.520300,49.758454,6,8,3,2019
9109,23197.0,23201.0,23252.0,23197.5,-0.03,23204.0,0.00,23204.0,23268.0,23213.428571,23214.533333,17.887213,48.130841,0,8,3,2019


In [83]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Khởi tạo mô hình XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',  # Mục tiêu phân loại nhị phân
    eval_metric='logloss',       # Thước đo để đánh giá trong quá trình huấn luyện
    learning_rate=0.05,
    sub_sample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=1,
    use_label_encoder=False,     # Tránh cảnh báo Deprecation
    random_state=42
)

# Huấn luyện mô hình
# (Sử dụng dữ liệu chưa scale cho XGBoost là phổ biến và thường hiệu quả hơn)
xgb_model.fit(X_train, y_train)

# ⭐️ Lưu ý: Ta dùng X_train/y_train chưa scale cho XGBoost
# Ta sẽ dùng X_train_scaled/y_train cho các mô hình NN/LSTM sau này

/home/quang/Documents/usd-vnd-prediction/.venv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [20:16:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "sub_sample", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [84]:
# Dự đoán trên tập kiểm tra
y_pred = xgb_model.predict(X_test)

# Đánh giá hiệu suất
accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác (Accuracy) trên tập Test: {accuracy:.4f}")

# Báo cáo Phân loại (Classification Report)
print("\n--- Báo cáo Phân loại (Classification Report) ---")
print(classification_report(y_test, y_pred))

# Ma trận Nhầm lẫn (Confusion Matrix)
print("\n--- Ma trận Nhầm lẫn (Confusion Matrix) ---")
print(confusion_matrix(y_test, y_pred))

Độ chính xác (Accuracy) trên tập Test: 0.6721

--- Báo cáo Phân loại (Classification Report) ---
              precision    recall  f1-score   support

           0       0.75      0.80      0.78      1611
           1       0.43      0.36      0.39       667

    accuracy                           0.67      2278
   macro avg       0.59      0.58      0.58      2278
weighted avg       0.66      0.67      0.66      2278


--- Ma trận Nhầm lẫn (Confusion Matrix) ---
[[1293  318]
 [ 429  238]]
